In [1]:
!pip install transformers
!pip install langdetect
!pip install langid

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 30.4 MB/s 
     |████████████████████████████████| 596 kB 38.0 MB/s 
     |████████████████████████████████| 895 kB 42.7 MB/s 
     |████████████████████████████████| 77 kB 3.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 981 kB 5.1 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=631a1a9149707050fb8142dd6840830adf18cdfbc6964434e2a1d63fd653b84f
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect
     |████████████████████████████████| 1.9 MB 5.0 MB/s 
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941188 sha256=db09dea8073dfee2a6ff4a5e79070d3a869c8150c4e1b6dcedd99c3afcddb6d5
  

In [2]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=06f09c3f961b09a97129da791af6a5409d42cc0179ad56026254f99b62c708c0
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [3]:
## for data
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold
## for processing
import re
import nltk
nltk.download('stopwords')
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for w2v
import gensim
import gensim.downloader as gensim_api
## for bert
import transformers

import langdetect
import langid

import wget

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
pd.options.display.max_colwidth = 150

In [5]:
lst_dics = ["История, Религия", "Социология, бизнес", "Литература"]

dtf = pd.DataFrame(lst_dics)

In [6]:
dtf = pd.read_csv("names (2).csv")

In [7]:
dtf["0"]

0                                                                Археологическое и этнографическое исследование Южного Сахалина в конце XIX – начале XX вв.
1                                                                                                                                                 HSE CHEER
2                                                                                                              Дизайн контента для социальных сетей НИУ ВШЭ
3                                                                                                                Инфляция оценок: изучение мнения студентов
4                                                                                                               Колымский край как место памяти о советском
5                             Основные направления трансформации дипломатии в условиях информационной революции: от цифровой дипломатии к дипломатии данных
6                                                             От

In [8]:
#langid.set_languages(['ru', 'en'])
#lang, score = langid.classify(dtf[0].iloc[1])
#print(lang, score)

In [9]:
lst_stopwords = nltk.corpus.stopwords.words("russian")
#lst_stopwords

def preprocessing(text):
  text = text.split()

In [10]:
#model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)
#model.most_similar(positive=[u'политик'])

In [11]:
import zipfile
model_url = 'http://vectors.nlpl.eu/repository/20/213.zip'
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
ref = zipfile.ZipFile(model_file, 'r')
ref.extractall()

In [12]:
nlp = gensim.models.KeyedVectors.load("model.model")

In [13]:
#nlp = gensim_api.load("word2vec-ruscorpora-300")
#nlp = gensim_api.load("geowac_lemmas_none_fasttextskipgram_300_5_2020")

nlp.most_similar(["психология"], topn=10)

[('психолингвистика', 0.7906066179275513),
 ('нейропсихология', 0.7846694588661194),
 ('парапсихология', 0.7698875665664673),
 ('педагогика', 0.7587850689888),
 ('социология', 0.7499883770942688),
 ('культурология', 0.7364685535430908),
 ('сексология', 0.7282429337501526),
 ('психофизиология', 0.717481255531311),
 ('конфликтология', 0.7139410376548767),
 ('лингвистика', 0.7136410474777222)]

In [14]:
## Function to apply
def get_similar_words(lst_words, top, nlp):
    lst_out = lst_words
    for tupla in nlp.most_similar(lst_words, topn=top):
        lst_out.append(tupla[0])
    return list(set(lst_out))


In [16]:
tags = pd.read_csv("tags.csv")

In [17]:
## Create Dictionary {category:[keywords]}
dic_clusters = {}



dic_clusters["история"] = get_similar_words(['история'], 
                 top=40, nlp=nlp)
dic_clusters["дизайн"] = get_similar_words(['дизайн'], 
                 top=40, nlp=nlp)
dic_clusters["религия"] = get_similar_words(['религия'], 
                 top=40, nlp=nlp)
dic_clusters["фотография"] = get_similar_words(['фотография'], 
                 top=40, nlp=nlp)
dic_clusters["экономика"] = get_similar_words(['экономика'], 
                 top=40, nlp=nlp)
dic_clusters["психология"] = get_similar_words(['психология'], 
                 top=40, nlp=nlp)
dic_clusters["бизнес"] = get_similar_words(['бизнес'], 
                 top=40, nlp=nlp)
dic_clusters["социология"] = get_similar_words(['социология'], 
                 top=40, nlp=nlp)

In [18]:
#RUS
tokenizer = transformers.BertTokenizer.from_pretrained('DeepPavlov/bert-base-multilingual-cased-sentence', do_lower_case=True)

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

'DeepPavlov/rubert-base-cased-sentence'

DeepPavlov/bert-base-multilingual-cased-sentence

DeepPavlov/bert-base-multilingual-cased-sentence

In [19]:
#RUS
nlp = transformers.TFBertModel.from_pretrained('DeepPavlov/bert-base-multilingual-cased-sentence', from_pt=True)
#nlp = transformers.TFBertModel.from_pretrained('DeepPavlov/rubert-base-cased', from_pt=True)

Downloading:   0%|          | 0.00/678M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [20]:
def utils_bert_embedding(txt, tokenizer, nlp):
    idx = tokenizer.encode(txt)
    idx = np.array(idx)[None,:]  
    embedding = nlp(idx)
    X = np.array(embedding[0][0][1:-1])
    return X
## create list of news vector
lst_mean_vecs = [utils_bert_embedding(txt, tokenizer, nlp).mean(0) 
                 for txt in lst_dics]
## create the feature matrix (n news x 768)
X = np.array(lst_mean_vecs)

In [21]:
# possible tags for the text
         
dic_y = [utils_bert_embedding(txt, tokenizer, nlp).mean(0) 
  for txt in dtf['0']]

In [22]:
#dic_clusters

In [23]:
#dic_y

In [24]:
similarities = np.array([metrics.pairwise.cosine_similarity(X, y.reshape(1,-1)).T.tolist()[0] for y in dic_y]).T
#similarities

In [25]:
## compute cosine similarities


#similarities = np.array([metrics.pairwise.cosine_similarity(X, y.reshape(1,-1)).T.tolist()[0] for y in dic_y.values()]).T


## adjust and rescale
labels = list(dtf['0'])
for i in range(len(similarities)):
    ### assign randomly if there is no similarity
    if sum(similarities[i]) == 0:
       similarities[i] = [0]*len(labels)
       similarities[i][np.random.choice(range(len(labels)))] = 1
    ### rescale so they sum = 1
    similarities[i] = similarities[i] / sum(similarities[i])

## classify the label with highest similarity score
predicted_prob = similarities
predicted = [labels[np.argmax(pred)] for pred in predicted_prob]

In [26]:
predicted_prob

array([[0.02000086, 0.03628409, 0.0276553 , 0.0269791 , 0.03186931,
        0.02416534, 0.01767767, 0.03621093, 0.02222334, 0.03033036,
        0.01746577, 0.0339037 , 0.02119551, 0.0318447 , 0.02216421,
        0.01483833, 0.03820598, 0.03582952, 0.01683335, 0.01577033,
        0.02854043, 0.0172554 , 0.02596906, 0.02506214, 0.02240111,
        0.02381106, 0.01755456, 0.02144844, 0.02493393, 0.02307204,
        0.02443615, 0.03830744, 0.01271577, 0.01979541, 0.01357739,
        0.01303878, 0.0210257 , 0.01620362, 0.01248916, 0.0277729 ,
        0.01821146, 0.03093036],
       [0.01596819, 0.02791043, 0.03344295, 0.02772313, 0.02251819,
        0.02656942, 0.02662488, 0.03129919, 0.02529232, 0.03188215,
        0.02821366, 0.02813528, 0.02210487, 0.02541574, 0.03014635,
        0.01474438, 0.03231554, 0.03805682, 0.01629692, 0.0182636 ,
        0.02820265, 0.01643673, 0.02640705, 0.0206971 , 0.02252021,
        0.02368088, 0.02027553, 0.01861329, 0.02216439, 0.02880945,
        0.02112

In [27]:
predicted

['Социальные сети: сопровождение и дизайн',
 'Цифровые платформы: право и экономика']

In [28]:
fin = pd.DataFrame(predicted)

In [29]:
fin2 = pd.DataFrame(lst_dics)

In [30]:
result = pd.concat([fin2, fin], axis=1)

In [31]:
result

,0,0
0,"История, Религия",Социальные сети: сопровождение и дизайн
1,"Социология, бизнес",Цифровые платформы: право и экономика
